In [4]:
# packages
import torch
import lightning as L
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings

# my model
from model.basicNN import L_NN_ID as iden

In [2]:
# model setting
model = iden(in_feature=4, h1=8, h2=9, out_feature=3)
torch.manual_seed = 81
epochs = 300
log_every_n_step = 10
batch_size = 32

In [6]:
# dataset
# import and modify data with pandas
directory = "data\iris\iris.csv"  # can use url
df = pd.read_csv(directory, delimiter=",")
print(df.head())

no_rep_variety = df["variety"].drop_duplicates()  # retrive none duplicated items
verbal_to_code = {
    verbal: i for i, verbal in enumerate(no_rep_variety)
}  # create verbal to code dictionary
df["variety"] = df["variety"].map(verbal_to_code)  # map/replace verbal with int

X = df.drop("variety", axis=1)  # axis=1 =remove column, axis=0 = remove 1st row/index
y = df["variety"]
# assigning inputs and labels and convert to tensor
input = torch.FloatTensor(X.values)
label = torch.LongTensor(y.values)  ##longTensor = int64

# train test split
train_input, test_input, train_label, test_label = train_test_split(
    input, label, test_size=0.2, random_state=81
)
train_dataset = TensorDataset(train_input, train_label)
test_dataset = TensorDataset(test_input, test_label)
# dataloader
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)
warnings.filterwarnings('ignore')

   sepal.length  sepal.width  petal.length  petal.width variety
0           5.1          3.5           1.4          0.2  Setosa
1           4.9          3.0           1.4          0.2  Setosa
2           4.7          3.2           1.3          0.2  Setosa
3           4.6          3.1           1.5          0.2  Setosa
4           5.0          3.6           1.4          0.2  Setosa


In [7]:
trainer = L.Trainer(max_epochs=epochs, log_every_n_steps=log_every_n_step)
# learning rate finder
tuner = L.pytorch.tuner.Tuner(trainer)
lr_find_results = tuner.lr_find(
    model,
    train_dataloaders=train_loader,
    min_lr=0.01,
    max_lr=1.0,
    early_stop_threshold=None,
)
new_lr = lr_find_results.suggestion()
model.learning_rate = new_lr
print(f"lr_find() suggest {new_lr:5f} for the learning rate.")
# train to fit
trainer.fit(model, train_dataloaders=train_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Ti Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Finding best initial lr: 100%|██████████| 100/100 [00:00<00:00, 111.27it/s]
Learning rate set to 0.06025595860743578
Restoring states from the checkpoint path at d:\Work repository\NN_test_build\.lr_find_2333803e-1ea3-4c44-b5db-08ef2a5d00ca.ckpt
Restored all states from the checkpoint at d:\Work repository\NN_test_build\.lr_find_2333803e-1ea3-4c44-b5db-08ef2a5d00ca.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type   | Params | Mode 
-------------

lr_find() suggest 0.060256 for the learning rate.
Epoch 299: 100%|██████████| 4/4 [00:00<00:00, 214.80it/s, v_num=4] 

`Trainer.fit` stopped: `max_epochs=300` reached.


Epoch 299: 100%|██████████| 4/4 [00:00<00:00, 155.22it/s, v_num=4]


In [10]:
#tensorboard loss result
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

Reusing TensorBoard on port 6006 (pid 8480), started 0:02:03 ago. (Use '!kill 8480' to kill it.)

In [11]:
# testing
with torch.no_grad():
    best_ckpt = trainer.checkpoint_callback.best_model_path
    tester = L.Trainer()
    tester.test(model, dataloaders=test_loader, ckpt_path=best_ckpt)
    correct = 0
    for i, data in enumerate(test_input):
        test_res = torch.tensor(model.forward(test_input))
        test_res_arg = torch.argmax(test_res, dim=1, keepdim=False)
        print(f"{i + 1}.) {test_res[i]} \t {test_res_arg[i]} \t {test_label[i]}")
        if test_res_arg[i] == test_label[i]:
            correct += 1
        right = model.calculate_accuracy(test_res, test_label)
        right
    print(f"{correct} / {len(test_label)} are correct")
    print(f"accuracy = {correct / len(test_label)}")


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at d:\Work repository\NN_test_build\lightning_logs\version_4\checkpoints\epoch=299-step=1200.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at d:\Work repository\NN_test_build\lightning_logs\version_4\checkpoints\epoch=299-step=1200.ckpt


Testing DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 184.37it/s]
1.) tensor([-72.5319,   2.3768,  10.4118]) 	 2 	 2
2.) tensor([ 12.0788,   4.2838, -12.7480]) 	 0 	 0
3.) tensor([-66.5345,   2.3664,   9.2107]) 	 2 	 2
4.) tensor([-26.0430,   3.8171,  -2.9796]) 	 1 	 1
5.) tensor([ 12.0788,   4.2838, -12.7480]) 	 0 	 0
6.) tensor([ 12.0788,   4.2838, -12.7480]) 	 0 	 0
7.) tensor([-29.7548,   3.6648,  -1.8109]) 	 1 	 1
8.) tensor([ 12.0788,   4.2838, -12.7480]) 	 0 	 0
9.) tensor([-57.9655,   2.5071,   7.0714]) 	 2 	 2
10.) tensor([ 12.0788,   4.2838, -12.7480]) 	 0 	 0
11.) tensor([-11.6837,   4.0228,  -6.7199]) 	 1 	 1
12.) tensor([-4.0109,  4.1071, -8.6662]) 	 1 	 1
13.) tensor([-93.4462,   2.4130,  14.6002]) 	 2 	 2
14.) tensor([-49.4859,   2.8551,   4.4015]) 	 2 	 2
15.) tensor([ 12.0788,   4.2838, -12.7480]) 	 0 	 0
16.) tensor([-65.9147,   2.3653,   9.0866]) 	 2 	 2
17.) tensor([ 12.0788,   4.2838, -12.7480]) 	 0 	 0
18.) tensor([ 12.0788,   4.2838, -12.7480]) 	 0 	 0
19.) te

In [12]:
# saving parameters
torch.save(model.state_dict(), "trained_parameters\l_id_iris.pt")

In [13]:
# region Identify by model
# load model and parameters
loaded_model = iden(in_feature=4, h1=8, h2=9, out_feature=3)
loaded_model.load_state_dict(torch.load("trained_parameters\l_id_iris.pt"))

# identify
with torch.no_grad():
    sus_input = torch.tensor(
        [[5, 3.5, 1.6, 0.6], [5.1, 2.5, 3, 1.1], [6.9, 3.2, 5.7, 2.3]]
    )
    print("the iris types are:")
    for i, data in enumerate(sus_input):
        sus_res = loaded_model.forward(sus_input).clone().detach()
        sus_res_arg = torch.argmax(sus_res, dim=1, keepdim=False)
        if sus_res_arg[i] == 0:
            sus = "Setosa"
        elif sus_res_arg[i] == 1:
            sus = "Versicolor"
        else:
            sus = "Virginica"
        print(f"{i + 1}.) {sus}")

the iris types are:
1.) Setosa
2.) Versicolor
3.) Virginica
